# Prepare val

For validation I will use: 
 * 25 last images from original train data
 * 25 images from the external data
 * 25 images from most confident preedictions in test
 
Everything is per class.

In [3]:
%matplotlib inline
from pylab import *
import cv2

In [4]:
from pathlib import Path
import numpy as np
import pandas as pd

In [5]:
data_path = Path('../data')

In [7]:
test_preds = pd.read_csv(str(data_path / 'Votings_stats.csv'))

In [8]:
test_preds.head()

,fname,arsenyinfo_top1,arsenyinfo_top2,bes_top1,bes_top2,mephistopheies_top1,mephistopheies_top2,cortwave_top1,cortwave_top2,ternaus,is_manip,best_model,votes
0,img_8f75e8d_manip.tif,LG-Nexus-5x,Motorola-Nexus-6,Motorola-Nexus-6,LG-Nexus-5x,Motorola-Nexus-6,Motorola-Nexus-6,LG-Nexus-5x,LG-Nexus-5x,Motorola-Nexus-6,manip,Motorola-Nexus-6,5
1,img_467a3e0_manip.tif,iPhone-4s,Motorola-Droid-Maxx,iPhone-4s,iPhone-4s,iPhone-4s,iPhone-4s,Motorola-Droid-Maxx,Motorola-Droid-Maxx,iPhone-4s,manip,iPhone-4s,6
2,img_eac0494_unalt.tif,Motorola-X,Motorola-X,Motorola-X,Motorola-X,Motorola-X,Motorola-X,Motorola-X,Motorola-X,Motorola-X,unalt,Motorola-X,9
3,img_de44aae_unalt.tif,Motorola-Droid-Maxx,Motorola-Droid-Maxx,Motorola-Droid-Maxx,Motorola-Droid-Maxx,Motorola-Droid-Maxx,Motorola-Droid-Maxx,Motorola-Droid-Maxx,Motorola-Droid-Maxx,Motorola-Droid-Maxx,unalt,Motorola-Droid-Maxx,9
4,img_c4d4fb0_unalt.tif,LG-Nexus-5x,LG-Nexus-5x,LG-Nexus-5x,LG-Nexus-5x,LG-Nexus-5x,LG-Nexus-5x,LG-Nexus-5x,LG-Nexus-5x,iPhone-6,unalt,LG-Nexus-5x,8


In [9]:
test_preds.shape

(2640, 13)

In [16]:
temp = []

for a, df in test_preds[['fname', 'is_manip', 'best_model', 'votes']].groupby(['best_model', 'is_manip']):
    temp += [df.sort_values('votes', ascending=False).head(25)]
    

In [19]:
test_preds_trunc = pd.concat(temp)

In [23]:
test_preds_trunc.head()

,fname,is_manip,best_model,votes
23,img_867a314_manip.tif,manip,HTC-1-M7,9
1498,img_10d4985_manip.tif,manip,HTC-1-M7,9
1708,img_d08048c_manip.tif,manip,HTC-1-M7,9
1673,img_103a8d3_manip.tif,manip,HTC-1-M7,9
1671,img_5cd9d50_manip.tif,manip,HTC-1-M7,9


In [27]:
test_preds_trunc['file_name'] = test_preds_trunc['fname'].apply(lambda x: (data_path / 'test' / x).absolute(), 1)

In [29]:
test_preds_trunc = test_preds_trunc.rename(columns={'best_model': 'target'})

In [30]:
test_preds_trunc.to_csv(str(data_path / 'test_preds_trunc.csv'), index=False)

# Now let's get last 5 from train

In [31]:
train_path = data_path / 'train'

In [52]:
train_file_names = list(train_path.glob('**/*.*'))

In [53]:
train_file_names = [x.absolute() for x in train_file_names]

In [54]:
df = pd.DataFrame({'file_name': train_file_names})

In [58]:
df['fname'] = df['file_name'].apply(lambda x: x.name, 1)

In [59]:
df['target'] = df['file_name'].apply(lambda x: x.parent.name, 1)

In [60]:
train_file_names[:2]

[PosixPath('/mnt/6e9adcc8-9cfe-4dbb-aaff-c2ae9c64a9bc/workspace/kaggle_camera/notebooks/../data/train/Sony-NEX-7/(Nex7)198.JPG'),
 PosixPath('/mnt/6e9adcc8-9cfe-4dbb-aaff-c2ae9c64a9bc/workspace/kaggle_camera/notebooks/../data/train/Sony-NEX-7/(Nex7)143.JPG')]

In [65]:
df.head()


,file_name,target,fname,ind
0,/mnt/6e9adcc8-9cfe-4dbb-aaff-c2ae9c64a9bc/work...,Sony-NEX-7,(Nex7)198.JPG,8
1,/mnt/6e9adcc8-9cfe-4dbb-aaff-c2ae9c64a9bc/work...,Sony-NEX-7,(Nex7)143.JPG,3
2,/mnt/6e9adcc8-9cfe-4dbb-aaff-c2ae9c64a9bc/work...,Sony-NEX-7,(Nex7)122.JPG,2
3,/mnt/6e9adcc8-9cfe-4dbb-aaff-c2ae9c64a9bc/work...,Sony-NEX-7,(Nex7)223.JPG,3
4,/mnt/6e9adcc8-9cfe-4dbb-aaff-c2ae9c64a9bc/work...,Sony-NEX-7,(Nex7)266.JPG,6


In [84]:
df['ind'] = df['fname'].str.extract('.*\)(\d+)\..*').astype(int)

/home/vladimir/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  """Entry point for launching an IPython kernel.


In [85]:
temp_val = []

for target, dft in df.groupby('target'):
    temp_val += [dft.sort_values(by='ind', ascending=False).head(25)]

In [86]:
df_train_val = pd.concat(temp_val)

In [92]:
df_train_train = df[df['ind'] <= 250]

In [93]:
df_train_val.shape

(250, 4)

In [94]:
df_train_train.shape

(2500, 4)

In [96]:
df_train_train.head()

,file_name,target,fname,ind
0,/mnt/6e9adcc8-9cfe-4dbb-aaff-c2ae9c64a9bc/work...,Sony-NEX-7,(Nex7)198.JPG,198
1,/mnt/6e9adcc8-9cfe-4dbb-aaff-c2ae9c64a9bc/work...,Sony-NEX-7,(Nex7)143.JPG,143
2,/mnt/6e9adcc8-9cfe-4dbb-aaff-c2ae9c64a9bc/work...,Sony-NEX-7,(Nex7)122.JPG,122
3,/mnt/6e9adcc8-9cfe-4dbb-aaff-c2ae9c64a9bc/work...,Sony-NEX-7,(Nex7)223.JPG,223
5,/mnt/6e9adcc8-9cfe-4dbb-aaff-c2ae9c64a9bc/work...,Sony-NEX-7,(Nex7)214.JPG,214


In [97]:
df_train_val.to_csv(str(data_path / 'val_df.csv'), index=False)

In [98]:
df_train_train.to_csv(str(data_path / 'train_df.csv'), index=False)

# Add first 25 images from flickr data

In [563]:
flickr_path = data_path / 'flickr_images'

In [564]:
flickr_file_names = list(flickr_path.glob('**/*.*'))

In [565]:
flickr_file_names = [x.absolute() for x in flickr_file_names]

In [566]:
flickr_file_names[:10]

[PosixPath('/mnt/6e9adcc8-9cfe-4dbb-aaff-c2ae9c64a9bc/workspace/kaggle_camera/notebooks/../data/flickr_images/Sony-NEX-7/24608015988_293dedc509_o.jpg'),
 PosixPath('/mnt/6e9adcc8-9cfe-4dbb-aaff-c2ae9c64a9bc/workspace/kaggle_camera/notebooks/../data/flickr_images/Sony-NEX-7/37112683936_07b43eb289_o.jpg'),
 PosixPath('/mnt/6e9adcc8-9cfe-4dbb-aaff-c2ae9c64a9bc/workspace/kaggle_camera/notebooks/../data/flickr_images/Sony-NEX-7/38685798051_21049c15dd_o.jpg'),
 PosixPath('/mnt/6e9adcc8-9cfe-4dbb-aaff-c2ae9c64a9bc/workspace/kaggle_camera/notebooks/../data/flickr_images/Sony-NEX-7/37610151464_0e87f5ec4a_o.jpg'),
 PosixPath('/mnt/6e9adcc8-9cfe-4dbb-aaff-c2ae9c64a9bc/workspace/kaggle_camera/notebooks/../data/flickr_images/Sony-NEX-7/27281298819_9785085140_o.jpg'),
 PosixPath('/mnt/6e9adcc8-9cfe-4dbb-aaff-c2ae9c64a9bc/workspace/kaggle_camera/notebooks/../data/flickr_images/Sony-NEX-7/38110820322_0d04b9202c_o.jpg'),
 PosixPath('/mnt/6e9adcc8-9cfe-4dbb-aaff-c2ae9c64a9bc/workspace/kaggle_camera/note

In [567]:
good_files = pd.read_csv(str(data_path / 'good_jpgs'), header=None)

In [568]:
good_files['target'] = good_files[0].str.split('/').str.get(1)

In [569]:
good_files['target'].unique()

array(['iphone_6', 'nexus_6', 'nexus_5x', 'moto_maxx', 'htc_m7',
       'iphone_4s', 'sony_nex7', 'samsung_s4', 'samsung_note3'], dtype=object)

In [570]:
good_files.head()

,0,target
0,./iphone_6/25563861418_8d7f66072a_o.jpg,iphone_6
1,./iphone_6/24568168517_2156a82e69_o.jpg,iphone_6
2,./iphone_6/39399094622_a1b6a0e180_o.jpg,iphone_6
3,./iphone_6/27636118319_f2a473fe32_o.jpg,iphone_6
4,./iphone_6/25537289038_bcf25bef2b_o.jpg,iphone_6


In [571]:
good_files['fname'] = good_files[0].apply(lambda x: x.split('/')[-1], 1)

In [572]:
good_files['file_name'] = good_files.apply(lambda x: (flickr_path / x['target'] / x['fname']).absolute(), 1)

In [573]:
# There is no Motorola X!

In [574]:
map_phone = {'iphone_6': 'iPhone-6', 
             'nexus_6': 'Motorola-Nexus-6', 
             'nexus_5x': 'LG-Nexus-5x', 
             'moto_maxx': 'Motorola-Droid-Maxx', 
             'htc_m7': 'HTC-1-M7',
             'iphone_4s': 'iPhone-4s', 
             'sony_nex7': 'Sony-NEX-7', 
             'samsung_s4': 'Samsung-Galaxy-S4', 
             'samsung_note3': 'Samsung-Galaxy-Note3'}

In [575]:
good_files['target'] = good_files['target'].apply(lambda x: map_phone[x])

In [576]:
good_files['target'].unique()

array(['iPhone-6', 'Motorola-Nexus-6', 'LG-Nexus-5x',
       'Motorola-Droid-Maxx', 'HTC-1-M7', 'iPhone-4s', 'Sony-NEX-7',
       'Samsung-Galaxy-S4', 'Samsung-Galaxy-Note3'], dtype=object)

In [577]:
df_flickr = pd.DataFrame({'file_name': flickr_file_names})

In [578]:
df_flickr['target'] = df_flickr['file_name'].apply(lambda x: x.parent.name, 1)

In [579]:
df_flickr['fname'] = df_flickr['file_name'].apply(lambda x: x.name, 1)

In [580]:
df_flickr.shape

(11603, 3)

In [581]:
Moto_X = df_flickr[df_flickr['target'] == 'Motorola-X']

In [582]:
to_good_moto_x = Moto_X.head(25)

In [583]:
to_good_moto_x.head()

,file_name,target,fname
9981,/mnt/6e9adcc8-9cfe-4dbb-aaff-c2ae9c64a9bc/work...,Motorola-X,25621489482_cd84ca566c_o.jpg
9982,/mnt/6e9adcc8-9cfe-4dbb-aaff-c2ae9c64a9bc/work...,Motorola-X,27758303644_9656b80b76_o.jpg
9983,/mnt/6e9adcc8-9cfe-4dbb-aaff-c2ae9c64a9bc/work...,Motorola-X,25621500042_19f0fd826b_o.jpg
9984,/mnt/6e9adcc8-9cfe-4dbb-aaff-c2ae9c64a9bc/work...,Motorola-X,26044092750_426d737083_o.jpg
9985,/mnt/6e9adcc8-9cfe-4dbb-aaff-c2ae9c64a9bc/work...,Motorola-X,24671017261_c87b625361_o.jpg


In [584]:
to_good_moto_x.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25 entries, 9981 to 10005
Data columns (total 3 columns):
file_name    25 non-null object
target       25 non-null object
fname        25 non-null object
dtypes: object(3)
memory usage: 800.0+ bytes


In [585]:
df_flickr[~df_flickr.index.isin(to_good_moto_x.index)].shape

(11578, 3)

In [586]:
df_flickr.head()

,file_name,target,fname
0,/mnt/6e9adcc8-9cfe-4dbb-aaff-c2ae9c64a9bc/work...,Sony-NEX-7,24608015988_293dedc509_o.jpg
1,/mnt/6e9adcc8-9cfe-4dbb-aaff-c2ae9c64a9bc/work...,Sony-NEX-7,37112683936_07b43eb289_o.jpg
2,/mnt/6e9adcc8-9cfe-4dbb-aaff-c2ae9c64a9bc/work...,Sony-NEX-7,38685798051_21049c15dd_o.jpg
3,/mnt/6e9adcc8-9cfe-4dbb-aaff-c2ae9c64a9bc/work...,Sony-NEX-7,37610151464_0e87f5ec4a_o.jpg
4,/mnt/6e9adcc8-9cfe-4dbb-aaff-c2ae9c64a9bc/work...,Sony-NEX-7,27281298819_9785085140_o.jpg


In [587]:
good_files.shape

(5377, 4)

In [588]:
good_files = pd.concat([good_files, to_good_moto_x]).reset_index(drop=True)

In [589]:
good_files.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5402 entries, 0 to 5401
Data columns (total 4 columns):
0            5377 non-null object
target       5402 non-null object
fname        5402 non-null object
file_name    5402 non-null object
dtypes: object(4)
memory usage: 168.9+ KB


In [590]:
# Let's pick 25 last images from all good files per class

In [591]:
temp_flickr = []

for target, dft in good_files.groupby('target'):
    temp_flickr += [dft.sort_values(by='fname', ascending=False).head(25)]

flickr_val = pd.concat(temp_flickr)

In [592]:
flickr_val.shape

(250, 4)

In [593]:
flickr_val.to_csv(str(data_path / 'flickr_val.csv'), index=False)

In [594]:
flickr_val.head()

,0,target,fname,file_name
2769,./htc_m7/38859029272_1b16047636_o.jpg,HTC-1-M7,38859029272_1b16047636_o.jpg,/mnt/6e9adcc8-9cfe-4dbb-aaff-c2ae9c64a9bc/work...
2161,./htc_m7/38585002062_806722481d_o.jpg,HTC-1-M7,38585002062_806722481d_o.jpg,/mnt/6e9adcc8-9cfe-4dbb-aaff-c2ae9c64a9bc/work...
2515,./htc_m7/38571347432_dfc8ca35cf_o.jpg,HTC-1-M7,38571347432_dfc8ca35cf_o.jpg,/mnt/6e9adcc8-9cfe-4dbb-aaff-c2ae9c64a9bc/work...
2881,./htc_m7/38560679106_3c868713f8_o.jpg,HTC-1-M7,38560679106_3c868713f8_o.jpg,/mnt/6e9adcc8-9cfe-4dbb-aaff-c2ae9c64a9bc/work...
2402,./htc_m7/38382462172_f1183ecc50_o.jpg,HTC-1-M7,38382462172_f1183ecc50_o.jpg,/mnt/6e9adcc8-9cfe-4dbb-aaff-c2ae9c64a9bc/work...


In [595]:
flickr_val.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 250 entries, 2769 to 64
Data columns (total 4 columns):
0            225 non-null object
target       250 non-null object
fname        250 non-null object
file_name    250 non-null object
dtypes: object(4)
memory usage: 9.8+ KB


In [596]:
flickr_val['id'] = flickr_val['target'] + '_' + flickr_val['fname']

In [597]:
df_flickr['id'] = df_flickr['target'] + '_' + df_flickr['fname']

In [598]:
good_files.head()

,0,target,fname,file_name
0,./iphone_6/25563861418_8d7f66072a_o.jpg,iPhone-6,25563861418_8d7f66072a_o.jpg,/mnt/6e9adcc8-9cfe-4dbb-aaff-c2ae9c64a9bc/work...
1,./iphone_6/24568168517_2156a82e69_o.jpg,iPhone-6,24568168517_2156a82e69_o.jpg,/mnt/6e9adcc8-9cfe-4dbb-aaff-c2ae9c64a9bc/work...
2,./iphone_6/39399094622_a1b6a0e180_o.jpg,iPhone-6,39399094622_a1b6a0e180_o.jpg,/mnt/6e9adcc8-9cfe-4dbb-aaff-c2ae9c64a9bc/work...
3,./iphone_6/27636118319_f2a473fe32_o.jpg,iPhone-6,27636118319_f2a473fe32_o.jpg,/mnt/6e9adcc8-9cfe-4dbb-aaff-c2ae9c64a9bc/work...
4,./iphone_6/25537289038_bcf25bef2b_o.jpg,iPhone-6,25537289038_bcf25bef2b_o.jpg,/mnt/6e9adcc8-9cfe-4dbb-aaff-c2ae9c64a9bc/work...


In [599]:
a = pd.DataFrame({'id': flickr_val['id'].values, 'is_val': flickr_val['file_name'].notnull()})

In [600]:
a.head()

,id,is_val
2769,HTC-1-M7_38859029272_1b16047636_o.jpg,True
2161,HTC-1-M7_38585002062_806722481d_o.jpg,True
2515,HTC-1-M7_38571347432_dfc8ca35cf_o.jpg,True
2881,HTC-1-M7_38560679106_3c868713f8_o.jpg,True
2402,HTC-1-M7_38382462172_f1183ecc50_o.jpg,True


In [601]:
df_flickr[df_flickr['target'] == 'Motorola-X'].head()

,file_name,target,fname,id
9981,/mnt/6e9adcc8-9cfe-4dbb-aaff-c2ae9c64a9bc/work...,Motorola-X,25621489482_cd84ca566c_o.jpg,Motorola-X_25621489482_cd84ca566c_o.jpg
9982,/mnt/6e9adcc8-9cfe-4dbb-aaff-c2ae9c64a9bc/work...,Motorola-X,27758303644_9656b80b76_o.jpg,Motorola-X_27758303644_9656b80b76_o.jpg
9983,/mnt/6e9adcc8-9cfe-4dbb-aaff-c2ae9c64a9bc/work...,Motorola-X,25621500042_19f0fd826b_o.jpg,Motorola-X_25621500042_19f0fd826b_o.jpg
9984,/mnt/6e9adcc8-9cfe-4dbb-aaff-c2ae9c64a9bc/work...,Motorola-X,26044092750_426d737083_o.jpg,Motorola-X_26044092750_426d737083_o.jpg
9985,/mnt/6e9adcc8-9cfe-4dbb-aaff-c2ae9c64a9bc/work...,Motorola-X,24671017261_c87b625361_o.jpg,Motorola-X_24671017261_c87b625361_o.jpg


In [602]:
flickr_val['id'] = flickr_val['id'].astype(str)
df_flickr['id'] = df_flickr['id'].astype(str)

In [603]:
train_good = df_flickr.merge(a, on='id', how='left')

In [604]:
sorted(df_flickr['target'].unique())

['HTC-1-M7',
 'LG-Nexus-5x',
 'Motorola-Droid-Maxx',
 'Motorola-Nexus-6',
 'Motorola-X',
 'Samsung-Galaxy-Note3',
 'Samsung-Galaxy-S4',
 'Sony-NEX-7',
 'iPhone-4s',
 'iPhone-6']

In [605]:
sorted(flickr_val['target'].unique())

['HTC-1-M7',
 'LG-Nexus-5x',
 'Motorola-Droid-Maxx',
 'Motorola-Nexus-6',
 'Motorola-X',
 'Samsung-Galaxy-Note3',
 'Samsung-Galaxy-S4',
 'Sony-NEX-7',
 'iPhone-4s',
 'iPhone-6']

In [606]:
df_flickr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11603 entries, 0 to 11602
Data columns (total 4 columns):
file_name    11603 non-null object
target       11603 non-null object
fname        11603 non-null object
id           11603 non-null object
dtypes: object(4)
memory usage: 362.7+ KB


In [607]:
train_good.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11603 entries, 0 to 11602
Data columns (total 5 columns):
file_name    11603 non-null object
target       11603 non-null object
fname        11603 non-null object
id           11603 non-null object
is_val       250 non-null object
dtypes: object(5)
memory usage: 543.9+ KB


In [608]:
sorted(train_good.loc[train_good['is_val'].notnull(), 'target'].unique())

['HTC-1-M7',
 'LG-Nexus-5x',
 'Motorola-Droid-Maxx',
 'Motorola-Nexus-6',
 'Motorola-X',
 'Samsung-Galaxy-Note3',
 'Samsung-Galaxy-S4',
 'Sony-NEX-7',
 'iPhone-4s',
 'iPhone-6']

In [609]:
sorted(flickr_val['target'].unique())

['HTC-1-M7',
 'LG-Nexus-5x',
 'Motorola-Droid-Maxx',
 'Motorola-Nexus-6',
 'Motorola-X',
 'Samsung-Galaxy-Note3',
 'Samsung-Galaxy-S4',
 'Sony-NEX-7',
 'iPhone-4s',
 'iPhone-6']

In [610]:
train_good.head()

,file_name,target,fname,id,is_val
0,/mnt/6e9adcc8-9cfe-4dbb-aaff-c2ae9c64a9bc/work...,Sony-NEX-7,24608015988_293dedc509_o.jpg,Sony-NEX-7_24608015988_293dedc509_o.jpg,NaN
1,/mnt/6e9adcc8-9cfe-4dbb-aaff-c2ae9c64a9bc/work...,Sony-NEX-7,37112683936_07b43eb289_o.jpg,Sony-NEX-7_37112683936_07b43eb289_o.jpg,NaN
2,/mnt/6e9adcc8-9cfe-4dbb-aaff-c2ae9c64a9bc/work...,Sony-NEX-7,38685798051_21049c15dd_o.jpg,Sony-NEX-7_38685798051_21049c15dd_o.jpg,NaN
3,/mnt/6e9adcc8-9cfe-4dbb-aaff-c2ae9c64a9bc/work...,Sony-NEX-7,37610151464_0e87f5ec4a_o.jpg,Sony-NEX-7_37610151464_0e87f5ec4a_o.jpg,NaN
4,/mnt/6e9adcc8-9cfe-4dbb-aaff-c2ae9c64a9bc/work...,Sony-NEX-7,27281298819_9785085140_o.jpg,Sony-NEX-7_27281298819_9785085140_o.jpg,NaN


In [611]:
flickr_val_df = train_good[train_good['is_val'].notnull()]
flickr_train_df = train_good[train_good['is_val'].isnull()]

In [612]:
flickr_val_df.to_csv(str(data_path / 'flickr_val.csv'), index=False)

In [613]:
flickr_train_df.to_csv(str(data_path / 'flickr_train.csv'), index=False)

In [614]:
flickr_val_df.tail()

,file_name,target,fname,id,is_val
10001,/mnt/6e9adcc8-9cfe-4dbb-aaff-c2ae9c64a9bc/work...,Motorola-X,29825925680_1ecd29e75a_o.jpg,Motorola-X_29825925680_1ecd29e75a_o.jpg,True
10002,/mnt/6e9adcc8-9cfe-4dbb-aaff-c2ae9c64a9bc/work...,Motorola-X,25714194673_9529e6e39d_o.jpg,Motorola-X_25714194673_9529e6e39d_o.jpg,True
10003,/mnt/6e9adcc8-9cfe-4dbb-aaff-c2ae9c64a9bc/work...,Motorola-X,28324695451_f84b1bf471_o.jpg,Motorola-X_28324695451_f84b1bf471_o.jpg,True
10004,/mnt/6e9adcc8-9cfe-4dbb-aaff-c2ae9c64a9bc/work...,Motorola-X,26121344776_66f24de818_o.jpg,Motorola-X_26121344776_66f24de818_o.jpg,True
10005,/mnt/6e9adcc8-9cfe-4dbb-aaff-c2ae9c64a9bc/work...,Motorola-X,28650836986_8a82d620f5_o.jpg,Motorola-X_28650836986_8a82d620f5_o.jpg,True


In [615]:
flickr_val['target'].value_counts()

Motorola-Droid-Maxx     25
Motorola-Nexus-6        25
Sony-NEX-7              25
HTC-1-M7                25
iPhone-4s               25
Samsung-Galaxy-Note3    25
Samsung-Galaxy-S4       25
LG-Nexus-5x             25
Motorola-X              25
iPhone-6                25
Name: target, dtype: int64